In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker
from urllib.request import urlopen
from datetime import datetime
import xmltodict

In [2]:
file = 'sqlite://'

engine = create_engine(file, echo=False)
Session = sessionmaker(bind=engine)
session = Session
Base = declarative_base(engine)

In [3]:
class Parlamentar(Base):
    __tablename__ = 'parlamentar'
    id = Column(Integer, primary_key=True)
    codigo = Column(String(4))
    name = Column(String(70))
    fullname = Column(String(70))
    sexo = Column(String(1))
    tratamento = Column(String(5))
    urlfoto = Column(String(255))
    urlpagina = Column(String(255))
    email = Column(String(255))
    partido = Column(String(6))
    uf = Column(String(2))
    
    def __repr__(self):
        return self.fullname

    
class Mandato(Base):
    __tablename__ = 'mandato'
    id = Column(Integer, primary_key=True)
    parlamentar_id = Column(Integer, ForeignKey('parlamentar.id'))
    codigo = Column(String(3))
    uf = Column(String(2))
    num_legis1 = Column(String(5))
    inicio_legis1 = Column(DateTime)
    fim_legis1 = Column(DateTime)
    num_legis2 = Column(String(5))
    inicio_legis2 = Column(DateTime)
    fim_legis2 = Column(DateTime)
    desc_participacao = Column(String(30))
    nome_suplente1 = Column(String(70))
    codigo_suplente1 = Column(String(6))
    participacao_suplente1 = Column(String(20))
    nome_suplente2 = Column(String(70))
    codigo_suplente2 = Column(String(6))
    participacao_suplente2 = Column(String(20))

    
class Exercicio(Base):
    __tablename__ = 'exercicio'
    id = Column(Integer, primary_key=True)
    parlamentar_id = Column(Integer, ForeignKey('parlamentar.id'))
    mandato_id = Column(Integer, ForeignKey('mandato.id'))
    codigo = Column(String(6))
    inicio = Column(DateTime)
    fim = Column(DateTime)
    sigla_afastamento = Column(String(6))
    causa_afastamento = Column(String(255))   

# Criação das tabelas
Base.metadata.create_all()

In [4]:
path = 'http://legis.senado.gov.br/dadosabertos/senador/lista/atual'
data = xmltodict.parse(urlopen(path).read())
fmt = '%Y-%m-%d'

In [5]:
def save_exercicio(mdex, id_parlamentar, id_mandato):
    exercicio = Exercicio()
    exercicio.codigo = mdex['CodigoExercicio']
    exercicio.parlamentar_id = id_parlamentar
    exercicio.mandato_id = id_mandato
    exercicio.inicio = datetime.strptime(mdex['DataInicio'], fmt)
                            
    if 'DataFim' in mdex.keys():
        exercicio.fim = datetime.strptime(mdex['DataFim'], fmt)
        exercicio.sigla_afastamento = mdex['SiglaCausaAfastamento']
        exercicio.causa_afastamento = mdex['DescricaoCausaAfastamento']
        
    session = Session()
    session.add(exercicio)
    session.commit()
    session.close()

In [6]:
for p in data['ListaParlamentarEmExercicio']['Parlamentares']['Parlamentar']:
    parlamentar = Parlamentar()
    id = p['IdentificacaoParlamentar']
    parlamentar.codigo = id['CodigoParlamentar']
    parlamentar.name = id['NomeParlamentar']
    parlamentar.fullname = id['NomeCompletoParlamentar']
    parlamentar.sexo = id['SexoParlamentar']
    parlamentar.tratamento = id['FormaTratamento']
    parlamentar.urlfoto = id['UrlFotoParlamentar']
    parlamentar.urlpagina = id['UrlPaginaParlamentar']
    parlamentar.email = id['EmailParlamentar']
    parlamentar.partido = id['SiglaPartidoParlamentar']
    parlamentar.uf = id['UfParlamentar']
    
    # print ('Nome Paralamentar ===> ',parlamentar.name)
    
    session = Session()
    session.add(parlamentar)
    session.commit()
    id_parlamentar = parlamentar.id
    session.close()
   
    mandato = Mandato()
    md  = p['Mandato']
    md1 = md['PrimeiraLegislaturaDoMandato']
    md2 = md['SegundaLegislaturaDoMandato']
    mandato.codigo = md['CodigoMandato']
    mandato.uf = md['UfParlamentar']
       
    mandato.num_legis1 = md1['NumeroLegislatura']
    mandato.inicio_legis1 = datetime.strptime(md1['DataInicio'], fmt)
    mandato.fim_legis1 = datetime.strptime(md1['DataFim'], fmt)
                                           
    mandato.num_legis2 = md2['NumeroLegislatura']
    mandato.inicio_legis2 = datetime.strptime(md2['DataInicio'], fmt)
    mandato.fim_legis2 = datetime.strptime(md2['DataFim'], fmt)
    
    mds = md['Suplentes']
    for suplente in mds.keys():
        if type(mds[suplente])==list:
            mandato.nome_suplente1 = mds[suplente][0]['NomeParlamentar']
            mandato.codigo_suplente1 = mds[suplente][0]['CodigoParlamentar']
            mandato.participacao_suplente1 = mds[suplente][0]['DescricaoParticipacao']
            mandato.nome_suplente2 = mds[suplente][1]['NomeParlamentar']
            mandato.codigo_suplente2 = mds[suplente][1]['CodigoParlamentar']
            mandato.participacao_suplente2 = mds[suplente][1]['DescricaoParticipacao']
        else:
            mandato.nome_suplente1 = mds[suplente]['NomeParlamentar']
            mandato.codigo_suplente1 = mds[suplente]['CodigoParlamentar']
            mandato.participacao_suplente1 = mds[suplente]['DescricaoParticipacao']
    
    mandato.parlamentar_id = id_parlamentar
    
    session = Session()
    session.add(mandato)
    session.commit()
    id_mandato = mandato.id
    session.close()

    
    mde = md['Exercicios']
    for ex in mde.keys():
        if type(md['Exercicios'][ex])!=list:
            save_exercicio(md['Exercicios'][ex], id_parlamentar, id_mandato)                
        else:
            for e in md['Exercicios'][ex]:
                save_exercicio(e, id_parlamentar, id_mandato)

In [7]:
def get_parlamentar(_name, scope=None):
    session = Session()

    if scope is None:
        parlamentar = session.query(Parlamentar).filter_by(name=_name)[0]
        mandato = session.query(Mandato).filter_by(parlamentar_id=parlamentar.id)[0]
        ex = session.query(Exercicio).filter_by(parlamentar_id=parlamentar.id)
        exercicios = [e for e in ex]
        dictp = dict(parlamentar = parlamentar, mandato = mandato, exercicios = exercicios)
    else:
        dictp = session.query(Parlamentar).filter(Parlamentar.name.like('%'+_name+'%')).all()
    return dictp

In [8]:
print ('# Busca pelo nome exato')
print ('-----------------------')
pl = get_parlamentar('Humberto Costa')
print(pl['parlamentar'].fullname, ' | ',pl['mandato'].uf )

# Busca pelo nome exato
-----------------------
Humberto Sérgio Costa Lima  |  PE


In [9]:
print ('\n# Busca no estilo like')
print ('----------------------')
pl = get_parlamentar('Paulo', 'like')
print ('Resultado:')
for p in pl:
    print (p.fullname, ' | ', p.uf)


# Busca no estilo like
----------------------
Resultado:
Paulo Roberto Bauer  |  SC
Paulo Renato Paim  |  RS
Paulo Roberto Galvão da Rocha  |  PA
